In [6]:
import os
os.environ['PYTHONPATH']='./iuindycar'
import iuindycar.orchestrator as orc

In [7]:
namespace = 'indycar'
broker_name = 'apollo-mqtt'

## Deploying Broker (Pub/Sub)
We use pub/sub borker to publish telemetry data, anomaly detection results

In [8]:
orc.deploy_broker(broker_name, namespace)

Deleting service apollo-mqtt
Pod apollo-mqtt has been deleted
Creating POD apollo-mqtt
Pod creation for apollo-mqtt has been submitted. Pod will become ready in several seconds
Creating Service apollo-mqtt
Broker apollo-mqtt has been deployed.
	Inside cluster use: apollo-mqtt.indycar.svc.cluster.local:61613
	Outside the cluster use node-ip:31176
	username: admin, password: password


## Deploying Socket Server

Socket server streams the data.

To stream a different years data, another socket server instance should be run. Multiple socket servers can use the same broker, but input and output topics should be different. Or another broker instance can be run.

In [9]:
socket_server_name = "indycar-socketserver-2017"
log_file_folder = "/nfs/indycar/shared/log_files/Indy_500_2017"
log_file_name = "eRPGenerator_TGMLP_20170528_Indianapolis500_Race.log"
broker_address = "apollo-mqtt.indycar.svc.cluster.local:61613" # this is printed when the broker is deployed.
input_topic_prefix = "2017"
output_topic = "compact_topology_out"
num_cars = "33" # number of cars to stream
target_node_port = "5000" # it should be a non used port

orc.deploy_socket_server(socket_server_name, namespace, log_file_folder,
                         log_file_name,
                         broker_address, input_topic_prefix,
                         output_topic, num_cars, target_node_port,
                         cpu_limit="2", memory_limit="10G")

Deleting service indycar-socketserver-2017
Deleting deployment indycar-socketserver-2017
Creating Deployment indycar-socketserver-2017
Deployment for indycar-socketserver-2017 has been submitted. Deployment will become ready in several seconds
Creating Service indycar-socketserver-2017
SocketServer indycar-socketserver-2017 has been deployed.
	Inside cluster use: indycar-socketserver-2017.indycar.svc.cluster.local:5000
	Outside the cluster use node-ip:5000


## Deploy TensorFlow Serving

This will deploy TensorFlow Serving


In [10]:
serving_name = "tf-serving"
model_path = "/nfs/indycar/serving/models/"
model_name = "lstm_speed_model"
orc.deploy_tf_serving(serving_name, namespace, 8500, 8501, "/nfs/indycar/serving/models/", "lstm_speed_model")

Deleting service tf-serving
Deleting deployment tf-serving
Creating Deployment tf-serving
Deployment for tf-serving has been submitted. Deployment will become ready in several seconds
Creating Service tf-serving
TensorFlow Serving tf-serving has been deployed.
	Use: tf-serving.indycar.svc.cluster.local:8500 for the grpc and 8501 fot the rest api.
	Note that TFServing is not accessible from the outside with this configuration


## Anomaly Detection Deploy

We need to deploy anomaly detection tasks one by one. We can deploy different detection algorithms and use them.

In [12]:
detection_name = "ad-car88"
detection_script_folder = "/nfs/indycar/serving/scripts"
detection_script_name = "car_inference.py"
car_number = 88
serving_address = "tf-serving.indycar.svc.cluster.local:8501" # printed out above
input_topic = input_topic_prefix + str(car_number)
sequence_len = 80
orc.deploy_car_anomaly_detection (detection_name, namespace, detection_script_folder,
                                  detection_script_name, car_number, serving_address,
                                  broker_address, input_topic,
                                  broker_address, sequence_len, cpu_limit=1)

Creating POD ad-car88
Pod creation for ad-car88 has been submitted. Pod will become ready in several seconds
Anomaly Detection pod deployed for 201788 has been deployed.


## Deploying Dashboard (Website)

In [16]:
dashboard_name = "indycar-dashboard-2017"
socket_server_address= "node-ip" # this should be replaced with your actual ip
socket_server_port = "30760"
orc.deploy_dashboard (dashboard_name, namespace, socket_server_address, socket_server_port)

Deleting service indycar-dashboard-2017
Deleting deployment indycar-dashboard-2017
Creating Deployment indycar-dashboard-2017
Deployment for indycar-dashboard-2017 has been submitted. Deployment will become ready in several seconds
Creating Service indycar-dashboard-2017
IndyCar Dashboard indycar-dashboard-2017 has been deployed. It might take several minutes to run
	Use: indycar-dashboard-2017.indycar.svc.cluster.local:3000 for internal access
	Outside the cluster use node-ip:32658
